In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:
# Label File
pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
...,...,...,...
22419,p081,c9,img_56936.jpg
22420,p081,c9,img_46218.jpg
22421,p081,c9,img_25946.jpg
22422,p081,c9,img_67850.jpg


In [36]:
# Loading ImageDataGenrator
from keras.preprocessing.image import ImageDataGenerator

In [37]:
# Making an object of ImageData Genrator
imageGen = ImageDataGenerator(rescale=1.0/255)

In [38]:
# Getting Training Genrator 
train_gen=imageGen.flow_from_directory(
    '../input/state-farm-distracted-driver-detection/imgs/train',
    batch_size=32,
    target_size=(100,100),
    class_mode = 'categorical',
)

Found 22424 images belonging to 10 classes.


# Making our CNN Model

In [39]:
# Making our CNN Model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.models import Sequential
model = Sequential([
    Conv2D(32,(3,3),activation="relu",input_shape =(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(512,activation='relu'),
    Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']  )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              34669568  
_________________________________________________________________
dense_4 (Dense)              (None, 512)              

In [ ]:
# Fitting our model
model.fit_generator(train_gen,epochs=5)

Epoch 1/5
701/701 [==============================] - 120s 172ms/step - loss: 0.2444 - accuracy: 0.9227
Epoch 2/5
701/701 [==============================] - 120s 172ms/step - loss: 0.0334 - accuracy: 0.9900
Epoch 3/5
701/701 [==============================] - 122s 173ms/step - loss: 0.0269 - accuracy: 0.9924
Epoch 4/5
674/701 [===========================>..] - ETA: 4s - loss: 0.0270 - accuracy: 0.9922

In [ ]:
# Making an object of ImageData Genrator
imageGen = ImageDataGenerator(rescale=1.0/255)

# Making Test Folder For ImageDataGenerator

In [ ]:
# Making a test directory in output folder ( that can be readable and writeable )
import os
os.mkdir('./test')
os.mkdir('./test/all_classes')

In [ ]:
# Importing copyfile so copy the testing images from the input test folder to ouput test folder
from shutil import copyfile

In [ ]:
# Copying from the input test folder to ouput test folder
d = pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')
for row in d.values:
    file_name = row[0]
    copyfile( '../input/state-farm-distracted-driver-detection/imgs/test/'+file_name , './test/all_classes/'+file_name )

In [ ]:
# Getting Testing Genrator 
test_gen=imageGen.flow_from_directory(
    './test',
    class_mode=None,
    shuffle=False,
    target_size=(100,100),
)

In [ ]:
# Predicting from test Genrator 
test_gen.reset()
predictions = model.predict_generator(test_gen,verbose=1)

In [ ]:
# Checking the shape of prediction
predictions.shape

In [ ]:
# Converting the Prediction same as sample submission dataFrame
# Making the dictionary of The Predcition
d1 = pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')
d ={
    'img':list(d1.values[:,0]),
}
for i in range(0,10):
    d['c' + str(i)] = list(predictions[:,i])
d

In [ ]:
# Converting the Prediction to DataFrame
df = pd.DataFrame(d)
df

In [33]:
# Converting DataFrame into CSV File
df.to_csv('./Prediction.csv',index=False)